In [ ]:
from os import system

def trim_file(filename, start_pt, end_pt, outfile):
    system("ffmpeg -i %s -ss %s -to %s -c:v copy -c:a copy %s" % (filename, start_pt, end_pt, outfile))

In [ ]:
def update_vtt(vtt_filename, start_time, end_duration):
    output = []
    in_bounds = False
    with open(vtt_filename, "r") as vtt_file:
        for line in vtt_file:
            if "-->" in line:
                utterance_start = line.split("-->")[0].strip()
                utterance_start = utterance_start.split(":")
                utterance_start = int(utterance_start[0]) * 3600 + int(utterance_start[1]) * 60 + float(utterance_start[2])

                utterance_end = line.split("-->")[1].strip()
                utterance_end = utterance_end.split(":")
                utterance_end = int(utterance_end[0]) * 3600 + int(utterance_end[1]) * 60 + float(utterance_end[2])

                if utterance_start >= start_time:
                    in_bounds = True
                elif utterance_start > start_time + end_duration:
                    in_bounds = False
                    break
                else:
                    continue

                new_timestamp_start = utterance_start - start_time
                new_timestamp_end   = utterance_end  - start_time

                new_timestamp_start = "%02d:%02d:%02d" % (new_timestamp_start // 3600, (new_timestamp_start % 3600) // 60, new_timestamp_start % 60)
                new_timestamp_end   = "%02d:%02d:%02d" % (new_timestamp_end // 3600, (new_timestamp_end % 3600) // 60, new_timestamp_end % 60)

                line = "%s --> %s\n" % (new_timestamp_start, new_timestamp_end)
                output.append(line)

            elif in_bounds:
                output.append(line)

    output_file = vtt_filename.split(".")[0] + "_trimmed.vtt"
    with open(output_file, "w") as vtt_file:
        vtt_file.writelines(output)